In [ ]:
pip install simpletransformers sacrebleu

In [ ]:
from datasets import load_dataset

ds = load_dataset("lavita/AlpaCare-MedInstruct-52k")
ds

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 52002
    })
})

In [ ]:
dataset_split = ds['train'].train_test_split(test_size=0.15, seed=42)
dataset_split

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 44201
    })
    test: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 7801
    })
})

In [ ]:
train_dataset = dataset_split['train'].to_pandas()
test_dataset = dataset_split['test'].to_pandas()
train_dataset = train_dataset[['input', "output"]]
test_dataset = test_dataset[['input', "output"]]

In [ ]:
train_dataset.shape

(44201, 2)

In [ ]:
test_dataset.shape

(7801, 2)

In [ ]:
train_dataset = train_dataset[0:20000]
test_dataet = test_dataset[0:5000]

In [ ]:
train_dataset['prefix'] = ""
test_dataset['prefix'] = ""
train_dataset

,input,output,prefix
0,"""The patient is a 70-year-old man with a compl...","To conduct a patient interview for this case, ...",
1,<noinput>,There are several primary factors contributing...,
2,<no input>,If both parents are carriers of a recessive ge...,
3,"According to latest surveillance data, inciden...",According to the most recent surveillance data...,
4,A 5-year-old boy presents with recurrent infec...,"Based on the provided case description, the un...",
...,...,...,...
19995,A longtime chain smoker presents with progress...,The presenting symptomatology in this case inc...,
19996,A. Chronic Bronchitis B. Emphysema C. Pulmonar...,The patient's symptoms of increasing shortness...,
19997,Immuno-oncology has the potential to revolutio...,The latest research on immuno-oncology demonst...,
19998,Which is generally more effective for moderate...,"As a medical expert, I can provide information...",


In [ ]:
train_df = train_dataset.rename(columns={"input":"input_text", "output":"target_text"})
test_df = test_dataset.rename(columns={"input":"input_text", "output":"target_text"})

In [ ]:
train_df = train_df.dropna(subset=["input_text", "target_text"]).reset_index(drop=True)
test_df = test_df.dropna(subset=["input_text", "target_text"]).reset_index(drop=True)

train_df["input_text"] = train_df["input_text"].astype(str)
train_df["target_text"] = train_df["target_text"].astype(str)
test_df["input_text"] = test_df["input_text"].astype(str)
test_df["target_text"] = test_df["target_text"].astype(str)

In [ ]:
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

Train shape: (20000, 3)
Test shape: (7801, 3)


In [ ]:
import logging
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = T5Args()
model_args.max_seq_length = 192
model_args.train_batch_size = 8
model_args.eval_batch_size = 2
model_args.num_train_epochs = 5
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 1000
model_args.save_steps = 1000
model_args.save_eval_checkpoints = True
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = True
model_args.num_return_sequences = 1
model = T5Model("mt5", "google/mt5-small", args=model_args)


model.train_model(
    train_df,
    eval_data=test_df,
    output_dir="outputs",
)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


  0%|          | 0/20000 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/2500 [00:00<?, ?it/s]

...

Running Epoch 5 of 5:   0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/7801 [00:00<?, ?it/s]

  0%|          | 0/7801 [00:00<?, ?it/s]

  0%|          | 0/7801 [00:00<?, ?it/s]

(12500,
 {'global_step': [1000,
   2000,
   2500,
   3000,
   4000,
   5000,
   5000,
   6000,
   7000,
   7500,
   8000,
   9000,
   10000,
   10000,
   11000,
   12000,
   12500],
  'eval_loss': [2.678153848654183,
   2.3131827078425435,
   2.206112730307507,
   2.1193421427117527,
   2.0101825726457023,
   1.9686922913513194,
   1.9686922913513194,
   1.8756561600773862,
   1.8176429157561445,
   1.8007512429499437,
   1.7798781067216987,
   1.7480459147250154,
   1.7221581013164162,
   1.7221581013164162,
   1.6968042532073262,
   1.6754789124296678,
   1.6686696583293885],
  'train_loss': [3.7341134548187256,
   2.7162060737609863,
   2.5968029499053955,
   2.3971505165100098,
   2.3261451721191406,
   2.028780937194824,
   2.028780937194824,
   2.1391351222991943,
   1.9939128160476685,
   2.3006222248077393,
   2.2083213329315186,
   1.6466169357299805,
   1.7595744132995605,
   1.7595744132995605,
   1.6788448095321655,
   1.5488801002502441,
   1.873799443244934]})

In [14]:
results = model.eval_model(
    test_df,
    verbose=True)

print(results)


  0%|          | 0/7801 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3901 [00:00<?, ?it/s]

{'eval_loss': 1.6686696583293885}


In [16]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

trained_model = MT5ForConditionalGeneration.from_pretrained("outputs")
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.mt5.tokenization_mt5.MT5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [17]:
repo_name = "YeBhoneLin10/FTL-Capstone-v2"

trained_model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...2sx5pd5/model.safetensors:   0%|          | 11.6kB / 1.20GB            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  .../tmp2dx5f66k/spiece.model: 100%|##########| 4.31MB / 4.31MB            

CommitInfo(commit_url='https://huggingface.co/YeBhoneLin10/FTL-Capstone-v2/commit/301a9a8b4f786ce72d1072d633d1a34e26163e90', commit_message='Upload tokenizer', commit_description='', oid='301a9a8b4f786ce72d1072d633d1a34e26163e90', pr_url=None, repo_url=RepoUrl('https://huggingface.co/YeBhoneLin10/FTL-Capstone-v2', endpoint='https://huggingface.co', repo_type='model', repo_id='YeBhoneLin10/FTL-Capstone-v2'), pr_revision=None, pr_num=None)

###### Test case

In [18]:
from simpletransformers.t5 import T5Model

model = T5Model("mt5", "YeBhoneLin10/FTL-Capstone-v2")


config.json:   0%|          | 0.00/757 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/893 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MT5Tokenizer'. 
The class this function is called from is 'T5Tokenizer'.
